# 🌟 **Oguz Arslan MS Thesis** 

# ⚽ **Rocket League Prediction Model** 🚗

## 🎨 **Table of Contents** 
1. [Introduction](#introduction)
2. [Data Analysis](#data-analysis)
3. [Results](#results)
4. [Conclusion](#conclusion)

### <a id="introduction"></a> 🔍 **Introduction** 
Welcome to this notebook! Here we take the popular video game [Rocket League](https://en.wikipedia.org/wiki/Rocket_League) (RL) and train a goal scoring prediction model using past professional esports matches.

<div align="center">
    <img src="Images/Example Gameplay.gif" alt="GIF example gameplay" width="700"/>
</div>

#### 💬 **Dataset Description**
The dataset consists of sequences of snapshots of the state of a RL match, including position and velocity of all players and the ball, as well as extra information. The **goal** is to predict -- from a given snapshot in the game -- for each team, the probability that they will score within the next 10 seconds of game time.

The data was taken from professional RL matches. Each event consists of a chronological series of frames recorded at 10 frames per second. All events begin with a kickoff, and most end in one team scoring a goal, but some are truncated and end with no goal scored due to circumstances which can cause gameplay strategies to shift, for example 1) nearing end of regulation (where the game continues until the ball touches the ground) or 2) becoming non-competitive, e.g. one team winning by 3+ goals with little time remaining.

#### 🗃️ **Dataset Columns**
- **`game_num`** _(train only)_: Unique identifier for the game from which the event was taken.
- **`event_id`** _(train only)_: Unique identifier for the sequence of consecutive frames.
- **`event_time`** _(train only)_: Time in seconds before the event ended, either by a goal being scored or simply when we decided to truncate the timeseries if a goal was not scored.
- **`ball_pos_[xyz]`**: Ball's position as a 3D vector.
- **`ball_vel_[xyz]`**: Ball's velocity as a 3D vector.
- For `i` in `[0,6)`:
    - **`p{i}_pos_[xyz]`**: Player `i`'s position as a 3D vector.
    - **`p{i}_vel_[xyz]`**: Player `i`'s velocity as a 3D vector.
    - **`p{i}_boost`**: Player `i`'s boost remaining, in `[0, 100]`. A player can consume boost to substantially increase their speed, and is required to fly up into the `z` dimension (besides driving up a wall, or the small air gained by a jump).
    - All `p{i}` columns will be `NaN` if and only if the player is [demolished](https://rocketleague.fandom.com/wiki/Demolition) (destroyed by an enemy player; will respawn after 3 seconds).
    - Players 0, 1, and 2 make up team `A` and players 3, 4, and 5 make up team `B`.
    - The orientation vector of the player's car (which way the car is facing) does not necessarily match the player's velocity vector, and this dataset does not capture orientation data.
- For `i` in `[0, 6)`:
    - **`boost{i}_timer`**: Time in seconds until big boost orb `i` respawns, or `0` if it is available. Big boost orbs grant a full 100 boost to a player driving over it. The orb `(x, y)` locations are roughly `[ (-61.4, -81.9), (61.4, -81.9), (-71.7, 0), (71.7, 0), (-61.4, 81.9), (61.4, 81.9) ]` with `z = 0`. (Players can also gain boost from small boost pads across the map, but we do not capture those pads in this dataset.)
- **`player_scoring_next`** _(train only)_: Which player scores at the end of the current event, in `[0, 6)`, or `-1` if the event does not end in a goal.
- **`team_scoring_next`** _(train only)_: Which team scores at the end of the current event (`A` or `B`), or `NaN` if the event does not end in a goal.
- **`team_[A|B]_scoring_within_10sec`** _(train only)_: **[Target columns]** Value of `1` if `team_scoring_next == [A|B]` and `time_before_event` is in `[-10, 0]`, otherwise `0`.
- **`id`** _(test only)_: Unique identifier for each test row. The result should be a pair of `team_A_scoring_within_10sec` and `team_B_scoring_within_10sec` probability predictions ffor each `id`, where the predictions can range the real numbers from `[0, 1]`.

#### 🔢 **Useful Game Values**
For more information, check out [RLBot Wiki](https://wiki.rlbot.org/botmaking/useful-game-values/).

<img src="Images/Map Dimensions.png" alt="image" width="500"/> <img src="Images/Boost Pad Locations.png" alt="image" width="466"/> 

#### <a id="data-analysis"></a> 📊 **Data Analysis** 
In this section, we will dive deep into the data using various tools and techniques.

Importing required libraries

In [4]:
import pandas as pd

In [140]:
df = pd.read_csv(r'C:\Users\Arslan\Desktop\All Data\train_0.csv') # The data frame

#### ✏️ **Tutorial Block**

In [86]:
## Read headers
df.columns

## Read a selected column (from rows 0 to 4)
df['game_num'][0:5] 

## Read many selected columns (from rows 0 to 9)
df[['game_num', 'p1_pos_y', 'p1_boost']][0:10]

## Read a selected row
df.iloc[0]

## Read many selected rows
df.iloc[[1, 3, 5]] # or df.iloc[1:7]

## Read a specific location (15th row and 23rd column)
df.iloc[15,23]

-0.6418

In [13]:
## Iterating through rows 
for index, row in df.head(10).iterrows(): ## Loop through the first 10 rows
    print(index, row['game_num']) ## Print the index and game number for each row

0 1
1 1
2 1
3 1
4 1
5 1
6 1
7 1
8 1
9 1
